# Proyecto Final PLN | Clickbait

Se proponen dos tareas principalmente para este reto:

Tarea 1: Sobre la clasificación del tipo de spoiler: La entrada es la publicación de clickbait y el documento vinculado. La tarea es clasificar el tipo de spoiler que justifica la publicación de clickbait (ya sea "phrase", "passage", "multi"). Para cada entrada, se debe generar una salida como `{"uuid": "<UUID>", "spoilerType": "<SPOILER-TYPE>"}` donde `<SPOILER-TYPE>` es una `phrase`, `passage`, or `multi`.

Tarea 2: Sobre la generación de spoilers: La entrada es la publicación de clickbait y el documento vinculado (y, opcionalmente, el tipo de spoiler si su enfoque usa este campo). La tarea es generar el spoiler para la publicación de clickbait. Para cada entrada, se debe generar una salida como `{"uuid": "<UUID>", "spoiler": "<SPOILER>"}` donde `<SPOILER>` es el spoiler de la publicación clickbait.

## Sobre los datos

Los datos vienen en formato JSON Lines (.jsonl) donde cada línea contiene una publicación de clickbait y la versión limpiada manualmente del documento vinculado. Para cada línea, el objetivo es clasificar el tipo de spoiler necesario (tarea 1) y/o generar el spoiler (tarea 2).

Para cada entrada en el conjunto de datos de entrenamiento y validación, están disponibles los siguientes campos:

- `uuid`: El `uuid` de la entrada del conjunto de datos.
- `postText`: El texto de la publicación de clickbait que se va a estropear.
- `targetParagraphs`: El contenido principal de la página web enlazada para clasificar el tipo de spoiler (tarea 1) y generar el spoiler (tarea 2). Consiste en los párrafos del contenido principal extraído manualmente.
- `targetTitle`: El título de la página web enlazada para clasificar el tipo de spoiler (tarea 1) y generar el spoiler (tarea 2).
- `targetUrl`: La URL de la página web vinculada.
- `humanSpoiler`: El spoiler generado por humanos (abstractivo) para la publicación de clickbait de la página web vinculada. Este campo solo está disponible en el conjunto de datos de entrenamiento y validación (no durante la prueba).
- `spoiler`: El spoiler extraído por humanos para la publicación de clickbait de la página web vinculada. Este campo solo está disponible en el conjunto de datos de entrenamiento y validación (no durante la prueba).
- `spoilerPositions`: la posición del spoiler extraído por humanos para la publicación de clickbait de la página web vinculada. Este campo solo está disponible en el conjunto de datos de entrenamiento y validación (no durante la prueba).
etiquetas: el tipo de spoiler (puede ser "frase", "pasaje" o "múltiple") que se clasificará en la tarea 1 (clasificación del tipo de spoiler). Este campo solo está disponible en el conjunto de datos de entrenamiento y validación (no durante la prueba). ).
Algunos campos contienen metainformación adicional sobre la entrada, pero no se usan: `postId`, `postPlatform`, `targetDescription`, `targetKeywords`, `targetMedia`.
